In [1]:
# !pip install openpyxl

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("/data/Augmented_data.csv")
df["Surname"] = df["Surname"].progress_apply(lambda x: str(x).capitalize())
df.head()

OSError: [Errno 116] Stale file handle: '/data/Augmented_data.csv'

In [108]:
cus_st = {}
for cus, st in zip(df["CustomerId"],df["Geography"]):
    if cus not in cus_st:
        cus_st[cus] = [st]
    else:
        if st not in cus_st[cus]:
            cus_st[cus].append(st)

In [109]:
df["Geography"] =  df["CustomerId"].progress_apply(lambda x: cus_st[x][0])

100%|██████████| 332397/332397 [00:00<00:00, 972821.66it/s]


In [110]:
cities = pd.read_excel("US_cities.xlsx")
state_city = {}
for st, ct in zip(cities["state"], cities["city"]):
    if st not in state_city:
        state_city[st] = [ct]
    else:
        state_city[st].append(ct)
        
        
state_mapper = {}
for st in df["Geography"]:
    if st not in state_mapper:
        state_mapper[st] = np.random.choice(list(state_city.keys()))
        
city_mapper = {}
for cust_id, st in zip(df["CustomerId"],df["Geography"]):
    if cust_id not in city_mapper:
        city_mapper[cust_id] = np.random.choice(state_city[state_mapper[st]])

In [111]:
state_mapper

{'Germany': 'Georgia', 'France': 'South Carolina', 'Spain': 'Michigan'}

In [112]:
df["Geography"] = df["Geography"].progress_apply(lambda x: state_mapper[x])
df["City"] = df["CustomerId"].progress_apply(lambda x: city_mapper[x])
df["month"] = df["date"].progress_apply(lambda x: x.split("-")[1])
df["year"] = df["date"].progress_apply(lambda x: x.split("-")[-1])
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: round(x,0))
df["Account_type"] = df["Tenure"].progress_apply(lambda x: "Gold" if x > 15 else "Premium" if x <= 15 and x >=8 else "Normal")

100%|██████████| 332397/332397 [00:00<00:00, 1235234.03it/s]


In [113]:
df["Other Accounts"] = df["Tenure"].progress_apply(lambda x: "Yes" if x > 7 else "No")
df["Salary Account"] = df["Satisfaction Score"].progress_apply(lambda x: "Yes" if x >= 3 else "No" )
df["HasCrCard"] = df["HasCrCard"].progress_apply(lambda x: "Yes" if x == 1 else "No")
df["IsActiveMember"] = df["IsActiveMember"].progress_apply(lambda x: "Yes" if x == 1 else "No")
df["Exited"] = df["Exited"].progress_apply(lambda x: "Exited" if x == 1 else "Not Exited")
df["CreditScore"] = df["CreditScore"].progress_apply(lambda x: 350 if x <=350 else x)

100%|██████████| 332397/332397 [00:00<00:00, 1067449.21it/s]


In [114]:
def financial_year(x):
    mon = int(x[0])
    year = int(x[1])
    if year == 2020:
        if mon <=3:
            return "FY19-20"
        else:
            return "FY20-21"
    if year == 2021:
        if mon <=3:
            return "FY20-21"
        else:
            return "FY21-22"
    if year == 2022:
        if mon <=3:
            return "FY21-22"
        else:
            return "FY22-23"
        
    if year == 2023:
        if mon <=3:
            return "FY22-23"
        else:
            return "FY23-24"
    if year == 2024:
        if mon <=3:
            return "FY23-24"
        else:
            return "FY24-25"

df["Financial_year"] = df[["month", "year"]].progress_apply(financial_year, axis=1)

100%|██████████| 332397/332397 [00:02<00:00, 157823.92it/s]


In [115]:
def cs_category(x):
    if x > 800:
        return "Exceptional"
    if x > 740:
        return "Very Good"
    if x > 670:
        return "Good"
    if x > 580:
        return "Fair"
    else:
        return "poor"
df["CS_Category"] = df["CreditScore"].progress_apply(cs_category)

df["SS_Category"] = df["Satisfaction Score"].progress_apply(lambda x: "Poor" if x <3 else "Good" if x < 4 else "Very Good")

df["Quarter"] = df["month"].progress_apply(lambda x: "Q4" if int(x)<=3 else "Q1" if int(x)<=6 else "Q2" if int(x)<=9 else "Q3" )

month_dic = {'May':5, 'Oct':10, 'Nov':11, 'Feb':2, 'Apr':4, 'Jul':7, 'Jan':1, 'Sep':9, 'Jun':6,
       'Aug':8, 'Dec':12, 'Mar':3}
month_dic = {v:k for k,v in month_dic.items()}

df["month"] = df["month"].progress_apply(lambda x: month_dic[int(x)])

100%|██████████| 332397/332397 [00:00<00:00, 1133632.45it/s]


In [116]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,...,City,month,year,Account_type,Other Accounts,Salary Account,Financial_year,CS_Category,SS_Category,Quarter
0,15658241,Olmstead,350.0,Georgia,Female,81,48,438091.980941,2,Yes,...,Johns Creek,Feb,2023,Gold,Yes,No,FY22-23,poor,Poor,Q4
1,15627162,Blesing,615.0,Georgia,Male,27,6,155245.230616,1,Yes,...,Albany,Mar,2020,Normal,No,No,FY19-20,Fair,Poor,Q4
2,15986692,Leiva,665.0,Georgia,Male,35,7,157859.682813,2,Yes,...,Dunwoody,Oct,2020,Normal,No,Yes,FY20-21,Fair,Very Good,Q3
3,15443069,Jamieson,541.0,South Carolina,Male,36,7,0.000000,2,No,...,Florence,Oct,2020,Normal,No,Yes,FY20-21,poor,Very Good,Q3
4,15611786,Tsui,621.0,Michigan,Female,69,9,0.000000,1,No,...,East Lansing,Jan,2020,Premium,Yes,Yes,FY19-20,Fair,Very Good,Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332392,15308570,Daigle,450.0,Georgia,Female,43,14,313145.164928,1,Yes,...,Athens-Clarke County,Jan,2022,Premium,Yes,Yes,FY21-22,poor,Very Good,Q4
332393,15864602,Griffith,350.0,South Carolina,Male,52,33,684971.208280,2,Yes,...,Columbia,Jul,2022,Gold,Yes,No,FY22-23,poor,Poor,Q2
332394,15626485,Lu,595.0,South Carolina,Female,26,8,85203.608400,1,Yes,...,North Charleston,Jan,2020,Premium,Yes,No,FY19-20,Fair,Poor,Q4
332395,15984022,Battle,350.0,Georgia,Male,53,30,425846.088483,1,Yes,...,Athens-Clarke County,May,2022,Gold,Yes,Yes,FY22-23,poor,Very Good,Q1


In [117]:
df.columns

Index(['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age',
       'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited', 'Complain', 'Satisfaction Score',
       'Card Type', 'Point Earned', 'date', 'City', 'month', 'year',
       'Account_type', 'Other Accounts', 'Salary Account', 'Financial_year',
       'CS_Category', 'SS_Category', 'Quarter'],
      dtype='object')

In [118]:
df.rename(columns={'Exited': 'Customer Status'}, inplace=True)

In [120]:
df.to_csv("/data/bank_customer_churn_data.csv", index=False)

OSError: [Errno 116] Stale file handle: '/data'